In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from time import time
import numpy as np

from dataset.modelnet40 import ModelNet40
from torch.utils.data import DataLoader

import itertools
from sklearn.metrics import accuracy_score

from models.dgcnn import DGCNN_cls
from models.pointnet2 import PointNet2_cls


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PointNet2_cls().to(device)

/home/robust/diffusion-point-cloud/pretrained/pointnet2_cls_weights/best_model.pth


In [3]:
dset = ModelNet40(num_points=1024, scale_mode='unit_sphere', partition='test')
dloader = DataLoader(dset, batch_size=32)

In [4]:
true_labels = []
pred_labels = []
with torch.no_grad():
    for i, batch in enumerate(dloader):
        pc = batch["pointcloud"].to(device)
        true_label = batch["cate"]
        true_labels.append(true_label.flatten().tolist())
        logits = model(pc.permute(0,2,1)).cpu()
        pred_labels.append(torch.max(logits, 1)[1].tolist())
        # break

L1 xyz:torch.Size([32, 3, 512]), points:torch.Size([32, 128, 512])
L2 xyz:torch.Size([32, 3, 128]), points:torch.Size([32, 256, 128])
L3 xyz:torch.Size([32, 3, 1]), points:torch.Size([32, 1024, 1])
L1 xyz:torch.Size([32, 3, 512]), points:torch.Size([32, 128, 512])
L2 xyz:torch.Size([32, 3, 128]), points:torch.Size([32, 256, 128])
L3 xyz:torch.Size([32, 3, 1]), points:torch.Size([32, 1024, 1])
L1 xyz:torch.Size([32, 3, 512]), points:torch.Size([32, 128, 512])
L2 xyz:torch.Size([32, 3, 128]), points:torch.Size([32, 256, 128])
L3 xyz:torch.Size([32, 3, 1]), points:torch.Size([32, 1024, 1])
L1 xyz:torch.Size([32, 3, 512]), points:torch.Size([32, 128, 512])
L2 xyz:torch.Size([32, 3, 128]), points:torch.Size([32, 256, 128])
L3 xyz:torch.Size([32, 3, 1]), points:torch.Size([32, 1024, 1])
L1 xyz:torch.Size([32, 3, 512]), points:torch.Size([32, 128, 512])
L2 xyz:torch.Size([32, 3, 128]), points:torch.Size([32, 256, 128])
L3 xyz:torch.Size([32, 3, 1]), points:torch.Size([32, 1024, 1])
L1 xyz:tor

In [14]:
accuracy_score(list(itertools.chain.from_iterable(true_labels)), list(itertools.chain.from_iterable(pred_labels)))

0.9315235008103727

In [15]:

class PGD:
    r"""
    PGD in the paper 'Towards Deep Learning Models Resistant to Adversarial Attacks'
    [https://arxiv.org/abs/1706.06083]

    Distance Measure : Linf

    Arguments:
        model (nn.Module): model to attack.
        eps (float): maximum perturbation. (Default: 8/255)
        alpha (float): step size. (Default: 2/255)
        steps (int): number of steps. (Default: 10)
        random_start (bool): using random initialization of delta. (Default: True)

    Shape:
        - images: :math:`(N, C, H, W)` where `N = number of batches`, `C = number of channels`,        `H = height` and `W = width`. It must have a range [0, 1].
        - labels: :math:`(N)` where each value :math:`y_i` is :math:`0 \leq y_i \leq` `number of labels`.
        - output: :math:`(N, C, H, W)`.

    Examples::
        >>> attack = torchattacks.PGD(model, eps=8/255, alpha=1/255, steps=10, random_start=True)
        >>> adv_images = attack(images, labels)

    """

    def __init__(self, model, device=None, eps=8/255, alpha=2/255, steps=10, random_start=True):
        super().__init__('PGD', model, device)
        self.eps = eps
        self.alpha = alpha
        self.steps = steps
        self.random_start = random_start
        self.supported_mode = ['default', 'targeted']

    def forward(self, images, labels):
        r"""
        Overridden.
        """

        images = images.clone().detach().to(self.device)
        labels = labels.clone().detach().to(self.device)

        if self.targeted:
            target_labels = self.get_target_label(images, labels)

        loss = nn.CrossEntropyLoss()
        adv_images = images.clone().detach()

        if self.random_start:
            # Starting at a uniformly random point
            adv_images = adv_images + \
                torch.empty_like(adv_images).uniform_(-self.eps, self.eps)
            adv_images = torch.clamp(adv_images, min=0, max=1).detach()

        for _ in range(self.steps):
            adv_images.requires_grad = True
            outputs = self.get_logits(adv_images)

            # Calculate loss
            if self.targeted:
                cost = -loss(outputs, target_labels)
            else:
                cost = loss(outputs, labels)

            # Update adversarial images
            grad = torch.autograd.grad(cost, adv_images,
                                       retain_graph=False, create_graph=False)[0]

            adv_images = adv_images.detach() + self.alpha*grad.sign()
            delta = torch.clamp(adv_images - images,
                                min=-self.eps, max=self.eps)
            adv_images = torch.clamp(images + delta, min=0, max=1).detach()

        return adv_images